# Imports
---
Import new skis / snowboards to an SQL database

In [32]:
import json
import pymysql
import re

In [33]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../app/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

In [34]:
db = setupdb()
cursor = db.cursor()

In [35]:
params = {
    'size': ['size', 'length'],
    'nose width': ['nose width', 'tip width'],
    'waist width': ['waist width'],
    'tail width': ['tail width'],
    'sidecut': ['sidecut radius', 'turning radius', 'radius', 'sidecut'],
    'setback': ['stance setback', 'setback'],
    'effective edge': ['effective edge', 'running length']
}

In [36]:
values = {
    'size': '',
    'nose width': '',
    'waist width': '',
    'tail width': '',
    'sidecut': '',
    'setback': '',
    'effective edge': ''
}

In [240]:
# Extraction of data 
# from string format to a tabular structure
# --------------------------------------------------
def extract_data(table):
    # Remove units from headings
    while re.search(r'\(([a-zA-Z]+)\)', table) != None:
        match = re.search(r'\(([a-zA-Z]+)\)', table)
        substr = table[match.start():match.end()]
        table = table.replace(substr, '')
    '''    
    while ')' in table:
        table = table[:table.find('(')] + table[table.find(')') + 1:]
    '''    
        
    breakpoints = []
    
    # Find location of headings in table
    for i in params:
        # Check each param alias
        for j in params[i]:
            if j in table:
                breakpoints.append(table.find(j))
                break
        
        values[i] = []
        
    # Extract table data into rows
    rows = []
    breakpoints.sort()
    for i, b in enumerate(breakpoints):
        if i > 0:
            rows.append(table[breakpoints[i-1]:b])
            print(f"Row: {table[breakpoints[i-1]:b]}")
        if i == len(breakpoints) -1:
            rows.append(table[b:])
    
    
    # Remove lables from rows
    for row in rows:
        for i in params:
            for j in params[i]:
                if j in row:
                    values[i] = row.replace(j, '')
                    break
                    
    
    # Format table rows as lists
    for val in values:
        try:
            # \u0020\u200b\u002f\u0020
            
            values[val] = values[val].strip()
            values[val] = values[val].replace('\u0020\u200b\u002f\u0020', '/')
            values[val] = values[val].replace('\u0020\u200b\u200b\u002f\u0020', '/')
            values[val] = values[val].split() 
            
        except Exception as e:
            x = e
            
    
    # Truncate long rows to the correct size
    num_sizes = len(values['size'])
    for val in values:
        # Sidecuts can have more than one radius
        if val != "sidecut":
            values[val] = values[val][:num_sizes]

        # Remove rogue characters from row values
        if len(values[val]) > 0 and type(values[val][0]) == str:
            for x, v in enumerate(values[val]):
                values[val][x] = values[val][x].replace('\u200b', '')
        
        
        # Fill empty or missing data with null values
        for x in range(num_sizes - len(values[val])):
            
            values[val].append(0)
            
    
        
        
    return values

In [115]:
# Save skiboard in MySQL DB
# --------------------------------------------------
def save_skiboard(skiboard):
    try:
        sql = f"""INSERT INTO skiboards (
            url,
            brand,
            model,
            year,
            name,
            slug,
            category,
            family,
            description,
            stiffness,
            shape,
            flex_profile,
            camber_profile,
            camber_details,
            core,
            core_profiling,
            fibreglass,
            laminates,
            resin,
            base,
            edges,
            edge_tech,
            topsheet,
            sidewall,
            inserts,
            asym,
            weight,
            womens,
            youth
        ) VALUES (
            '{skiboard['url']}',
            '{skiboard['brand']}',
            '{skiboard['model']}',
            '{skiboard['year']}',
            '{str(skiboard['brand']).title()} {str(skiboard['model']).title()} {str(skiboard['year'])}',
            '{str(skiboard['brand']).lower().replace(' ', '-')}-{str(skiboard['model']).lower().replace(' ', '-')}-{str(skiboard['year'])}',
            '{skiboard['category']}',
            '{skiboard['family']}',
            '{skiboard['description']}',
            '{skiboard['stiffness']}',
            '{skiboard['shape']}',
            '{skiboard['flex_profile']}',
            '{skiboard['camber_profile']}',
            '{skiboard['camber_details']}',
            '{skiboard['core']}',
            '{skiboard['core_profiling']}',
            '{skiboard['fibreglass']}',
            '{skiboard['laminates']}',
            '{skiboard['resin']}',
            '{skiboard['base']}',
            '{skiboard['edges']}',
            '{skiboard['edge_tech']}',
            '{skiboard['topsheet']}',
            '{skiboard['sidewall']}',
            '{skiboard['inserts']}',
            '{skiboard['asym']}',
            '{skiboard['weight']}',
            '{skiboard['womens']}',
            '{skiboard['youth']}'
        )"""
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(f"ERROR:\n{e}")

In [39]:
# Save data in MySQL DB
# --------------------------------------------------
def save_values(skiboard, values):
    #print(f"Importing...\n{str(skiboard['brand']).capitalize()} {str(skiboard['model']).capitalize()} {str(skiboard['year'])}\nValues:\n{values}")

    for x in range(len(values['size'])):
        try:
            sql = f"""REPLACE INTO sizes (
                skiboard_id,
                size,
                nose_width,
                waist_width,
                tail_width,
                sidecut,
                setback,
                effective_edge
            ) VALUES (
                '{str(skiboard['id'])}',
                '{str(values['size'][x])}',
                '{float(values['nose width'][x])}',
                '{float(values['waist width'][x])}',
                '{float(values['tail width'][x])}',
                '{str(values['sidecut'][x])}',
                '{float(values['setback'][x])}',
                '{float(values['effective edge'][x])}'
            )"""

            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(f"ERROR: size ({values['size'][x]})\n{e}\n")
            print(f"SQL:\n{sql}\n\n")
    
    return True

# Create New SkiBoard
---

In [40]:
id_counter = 97

In [41]:
print(id_counter)

97


In [158]:
skiboard = {
    'id': id_counter,
    'url': '',
    'brand': 'Ride',
    'model': 'Back Talk',
    'year': 2020,
    'category': 'Snowboard',
    'family': '',
    'description': "Be careful with this board, disrespect it and you might get floored. Do right by it and you'll find yourself with a loyal mountain companion that'll stay by your side through thick powder and thin cover. The Ride Back Talk Snowboard is freestyle beast loaded with pop and power for high-flying rippers. The asymmetrical twin shape, quadratic sidecut, and stiff flex make this lightweight stick a twitchy fast, super responsive savage in the park and all over the frontside.".replace("'", "`"),
    'stiffness': 7,
    'shape': 'Twin',
    'flex_profile': '',
    'camber_profile': 'Hybrid Camber',
    'camber_details': '',
    'core': "Women's Specific Performance™ Core".replace("'", "`"),
    'core_profiling': '',
    'fibreglass': 'Hybrid Glass',
    'laminates': 'Carbon Array™ 5 Laminates ~ Double Impact Plates',
    'base': 'Sintered, Stone Ground Base',
    'edges': 'Cleave Edge™ Steel',
    'edge_tech': '',
    'topsheet': '',
    'resin': '',
    'sidewall': 'Roll-In Slimewall® ~ Carbon Slimewalls®',
    'inserts': '4x2',
    'asym': 1,
    'weight': 0,
    'womens': 1,
    'youth': 0
}

In [159]:
save_skiboard(skiboard)

In [160]:
print(skiboard['id'])

105


# Add Sizes to SkiBoard
---

In [205]:
skiboard = {
    'id': 105
}

In [241]:
table = input("Please submit your ski/snowboard size table\n\n").lower()

Please submit your ski/snowboard size table

Size (cm)	138	142	146	150 Effective Edge (mm)	1076	1101	1126	1152 Tip Width (mm)	279	282	285	289 Waist Width (mm)	236	238	240	242 Tail Width (mm)	279	282	285	289 Sidecut Radius (m)	(9.4​​/8.15) ​​/ (6.9​​/5.65) ​​/ (9.4​​/8.15)	(9.5​​/8.25) ​​/ (7​​/5.75) ​​/ (9.5​​/8.25)	(9.6​​/8.35) ​​/ (7.1​​/5.85) ​​/ (9.6​​/8.35)	(9.7​​/8.45) ​​/ (7.2​​/5.95) ​​/ (9.7​​/8.45) Stance Setback (mm)	0	0	0	0 Rider Weight (lbs)	60-100	70-100	85-130	100-150 Width	Regular	Regular	Regular	Regular


In [242]:
values = extract_data(table)

Row: size 	138	142	146	150 
Row: effective edge 	1076	1101	1126	1152 
Row: tip width 	279	282	285	289 
Row: waist width 	236	238	240	242 
Row: tail width 	279	282	285	289 
Row: sidecut radius 	(9.4​​/8.15) ​​/ (6.9​​/5.65) ​​/ (9.4​​/8.15)	(9.5​​/8.25) ​​/ (7​​/5.75) ​​/ (9.5​​/8.25)	(9.6​​/8.35) ​​/ (7.1​​/5.85) ​​/ (9.6​​/8.35)	(9.7​​/8.45) ​​/ (7.2​​/5.95) ​​/ (9.7​​/8.45) 


In [243]:
for row in values:
    print(f"{row}:\n{values[row]}\n\n")

size:
['138', '142', '146', '150']


nose width:
['279', '282', '285', '289']


waist width:
['236', '238', '240', '242']


tail width:
['279', '282', '285', '289']


sidecut:
['(9.4/8.15)/(6.9/5.65)/(9.4/8.15)', '(9.5/8.25)/(7/5.75)/(9.5/8.25)', '(9.6/8.35)/(7.1/5.85)/(9.6/8.35)', '(9.7/8.45)/(7.2/5.95)/(9.7/8.45)']


setback:
['0', '0', '0', '0']


effective edge:
['1076', '1101', '1126', '1152']




In [209]:
success = save_values(skiboard, values)
if success:
    print("\nSUCCESS!")
    id_counter += 1


SUCCESS!


In [238]:
table = "Length (cm) (cm)123 123 123 (12.3)"

match = re.search(r'\(([a-zA-Z]+)\)', table)
if match != None:
    print(match.start())
    print(match.end())


7
11


In [239]:
while re.search(r'\(([a-zA-Z]+)\)', table) != None:
    match = re.search(r'\(([a-zA-Z]+)\)', table)
    substr = table[match.start():match.end()]
    print(f"Substr: {substr}")
    table = table.replace(substr, '')

print(table)

Substr: (cm)
Length  123 123 123 (12.3)


In [ ]:
if re.search(r'\(([a-zA-Z])\)', table):
    print("OK")
    x = re.search("(\w)", table)
    substr = table[x.start():x.end()]
    print(f"SubStr: {substr}")
    table.replace(substr, '')
    print(table)
else:
    print("Done")

In [ ]:
print(x)